# Hate Speech Detection

I want to build an algorithim to  categorize tweets.  This will use the Hate Speach and Offensive Language dataset at [https://www.kaggle.com/datasets/mrmorj/hate\-speech\-and\-offensive\-language\-dataset?select=labeled\_data.csv](https://www.kaggle.com/datasets/mrmorj/hate-speech-and-offensive-language-dataset?select=labeled_data.csv).  This dataset is 24782 tweets long with users voting on if the tweet is offensive 0, hate speach 1 or neither 2, with the winning label being selected as a class.  

## Goal

My goal is to identify groupings of tweets based on topic.  With 27k+  tweets, I should be able to find 30-50 groups based on topics.

## Approach

I am planning to use a combination of TF-IDF, matrix factorization, and clustering algorithims to tweets by 1 and 2 word clusters.  I will then create similarity tables and use rainbow clustering to find groups of tweets.

### Disclaimer

This assignment necessary contains hate speach, and offensive langauge.  This does not represent the views of the author.



In [23]:
#General Imports
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import statistics
from collections import Counter
#EDA Imports
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from random import sample
from sklearn.metrics import recall_score, precision_score, f1_score

#Model build Imports
from sklearn.cluster import DBSCAN

In [24]:
#Initial Import
raw_data = pd.read_csv('Data/labeled_data.csv', header = 0)
print(raw_data)
raw_data = raw_data.drop(columns = 'Unnamed: 0')
print(statistics.mean(raw_data['tweet'].str.len()))

       Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0               0      3            0                   0        3      2   
1               1      3            0                   3        0      1   
2               2      3            0                   3        0      1   
3               3      3            0                   2        1      1   
4               4      6            0                   6        0      1   
...           ...    ...          ...                 ...      ...    ...   
24778       25291      3            0                   2        1      1   
24779       25292      3            0                   1        2      2   
24780       25294      3            0                   3        0      1   
24781       25295      6            0                   6        0      1   
24782       25296      3            0                   0        3      2   

                                                   tweet  
0      !!! RT @m

## Import and Cleaning

### Import

This data imports fairly easily.  I donloaded a copy of the file from kaggle and mirrored it to my GitHub so I could autodownload it for your use.

### Data Description
The data concists of 24782 tweets which have been user labeled as either hate, offensive, or neither, along with a total cont of votes.

### Cleaning

#### **Tweets with newline**

There appears to be a mismatch between the number of lines generated and the number of lines imported.  This is caused by newline characters, commas, quotation marks, and other supidity mixed in.  I chose to build my own parser to deal with all the issues and allow me to deal with the various problems with the data.\
**Update**\
There are skipped tweet numbers with no indicator in the dataset other than looking.

##### **Some examples of problematic formating.**

22422,3,0,3,0,1,"This bitch fell straight through the chair... Im high, of course I laughed..\
\
\
\
Even if I wasn't I would've still laughed."

256,3,0,3,0,1,"""@TheNewSAT: #NewSATQuestions\
Yeah bitch, yeah bitch, call me _______:\
a.) Maybe\
b.) Steve-O\
c.) Later\
d.) Jesse Pinkman""\
@machinegunkelly"

#### **Username Cleanup**

I chose to just drop all usernames.  This is due to usernames being random characters and it's out of scope for this project to try to do abusive username detection.

#### **Unicode Characters, numbers, and non-text characters**

I also chose to drop all unicode characters (Doing this save soo much time), numbers and most non-text characters.  The exception was the "*" character.  The * character is used to mask offensive words and when used, the resulting word length is fairly standard.

# Initial EDA
The average character count of the uncleaned tweets is 85.  The average word count is 14.  



In [25]:
print("Mean Charater Count: " + str(statistics.mean(raw_data['tweet'].str.len())))
print("Mean Word Count: " + str(np.mean(raw_data['tweet'].apply(lambda x: len([words for words in x.split(" ") if isinstance(x, str)])))))
print("Basic Data Charasitics" + str(Counter(raw_data['class'])))
print("30 Most common words.")
Counter(" ".join(raw_data['tweet']).split()).most_common(30)

Mean Charater Count: 85.43606504458701
Mean Word Count: 14.07097607230763
Basic Data CharasiticsCounter({1: 19190, 2: 4163, 0: 1430})
30 Most common words.


[('a', 9099),
 ('RT', 7539),
 ('bitch', 6638),
 ('the', 6590),
 ('I', 6472),
 ('to', 5240),
 ('you', 4881),
 ('and', 3670),
 ('that', 3111),
 ('my', 3072),
 ('in', 2902),
 ('is', 2759),
 ('bitches', 2576),
 ('like', 2534),
 ('of', 2503),
 ('on', 2361),
 ('be', 2304),
 ('me', 2249),
 ('for', 2023),
 ('hoes', 1925),
 ('with', 1778),
 ('pussy', 1711),
 ('this', 1575),
 ("I'm", 1496),
 ('hoe', 1470),
 ('ass', 1447),
 ('it', 1445),
 ('your', 1423),
 ('get', 1328),
 ('up', 1313)]

In [26]:
def text_cleanup(text):
    #Remove usernames replace @xxx: with whitespace
    text = re.sub("@.*?:", " ", text)
    #remove unicode characters
    text = re.sub("&#[0-9]{1,6};", " ", text)
    #lowercase all text
    text = text.lower()
    #remove URL's
    text = re.sub('http[s]?:\/\/.*?"', " ", text)
    #remove special characters and numberrs'
    #I chose to leave in *'s as they are a common manipulation to get around filters
    text = re.sub("[^a-z\*]", " ", text)
    #remove repeat whitespace and newlines
    text = re.sub("\s\s+", " ", text)
    text = re.sub("\n+", "", text)
    return text
raw_data["tweet"] = raw_data["tweet"].apply(text_cleanup)
print(raw_data['tweet'])

0         rt as a woman you shouldn t complain about cl...
1         rt boy dats cold tyga dwn bad for cuffin dat ...
2         rt you ever fuck a bitch and she start to cry...
3                     rt viva based she look like a tranny
4         rt the shit you hear about me might be true o...
                               ...                        
24778    you s a muthaf***in lie pearls corey emanuel r...
24779    you ve gone and broke the wrong heart baby and...
24780    young buck wanna eat dat nigguh like i aint fu...
24781                youu got wild bitches tellin you lies
24782     ruffled ntac eileen dahlia beautiful color co...
Name: tweet, Length: 24783, dtype: object


## EDA Procedure

### Basic Data Charaistics After Cleanup

A tweet at the time of this data being collected can contain at maxiumum 280 Characters.  
Mean Char Count
Mean Word Count

### Formatting Data

I will start with dropping all the non-tweet columns.  This will be blind learning.

### Bagging

I chose to use TF\-IDF for my bagging procedure.  I am testing using unigrams single words only.  I may retest this using bigrams later., and am removing common English stopwords.  I decided not to restrict common words as the most common words are 27%.  I may experiment with this later.  

Exploratory analays was done on unigrams and bigrams.  

### DBSCAN

DBSCAN or Density\-Based Spatial Clustering of Applications with Noise is a clustering algorithm that uses the density of datapoints to classify objects.  In this case, I'm using DBSCAN to provide coloring of my graphed results.  Although I will use it later for actual model building, I am using it here to provide color differences.


That's not good.  All of the data seems to be in 1 large clump.  That will not do for classification.  I can try to get rid of the outliers, or I can try to clean the data.  I will try both. 

## Attempt to get rid of highly common elements.

In this I will rerun the TF\-IDF with a maximum document frequency of .2 and .15 and see if I get a better spread.  This change reduced the spacing between the 2 sets of elements.  



In [27]:
def color_graph(model_data, plot_title = ""):\
    grid = sns.FacetGrid(data =model_data,  col = 'gram_size', row = 'k').set(title = plot_title)
    grid.map(sns.scatterplot(
        x=model_data['x'].tolist(), y=model_data['y'].tolist(),
        hue=model_data['predict'].tolist(),
        legend="full",
        alpha=0.3
    ))
    print(model_data['predict'].value_counts())
    return model_data

In [30]:
def TF_IDF(tweets, ngram_range, max_df):
    tfidf = TfidfVectorizer(input='content', encoding='utf-8', \
                            decode_error='strict', strip_accents='ascii', \
                            lowercase=True, preprocessor=None, tokenizer=None, \
                            analyzer='word', stop_words='english',  \
                            ngram_range=(ngram_range, ngram_range),max_df = max_df, max_features=None,\
                            vocabulary=None, binary=False, norm='l2', \
                            use_idf=True, smooth_idf=True, sublinear_tf=False)
    tfidf = tfidf.fit(raw_data['tweet'])
    tfidf_data = tfidf.transform(raw_data['tweet'])
    feature_names = tfidf.get_feature_names_out()
    return (tfidf_data, feature_names)

def TSNE_Process(tfidf_data, gram_size, k):
    tsne_results = TSNE(n_components=2, learning_rate='auto'\
                                 ,init='random', perplexity=30, n_iter = 600\
                                ).fit_transform(tfidf_data)
    tsne_results = pd.DataFrame(tsne_results, columns = ['x', 'y'])
    tsne_results['gram_size'] = gram_size
    tsne_results['k'] = k
    return tsne_results
def build_model(tsne_results):
    dbscan = DBSCAN(eps=2.0, min_samples=100,\
                    metric='euclidean', metric_params=None, \
                    algorithm='auto', leaf_size=30, p=None, \
                    n_jobs=None)
    model = dbscan.fit(tsne_results)
    model_data['predict'] = model.fit_predict(model_data,model_data)
    model_data['predict'] = model_data['predict'].where(model_data['predict'] == 0, 1)
    #print(model_data['predict'].value_counts())
    return model_data

In [31]:
model_data = pd.DataFrame()
unigram_k_1_TF_IDF, unigram_k_1_features = TF_IDF(raw_data['tweet'], 1, 1)
unigram_k_1_TSNE = TSNE_Process(unigram_k_1_TF_IDF, 'Unigram', 1)
model_data = pd.concat([build_model(unigram_k_1_TSNE), model_data])
unigram_k_15_TF_IDF, unigram_k_15_features = TF_IDF(raw_data['tweet'], 1, .15)
unigram_k_15_TSNE = TSNE_Process(unigram_k_15_TF_IDF, 'Unigram', .15)
model_data = pd.concat([build_model(unigram_k_15_TSNE), model_data])

bigram_k_1_TF_IDF, bigram_k_1_features = TF_IDF(raw_data['tweet'], 2, 1)
bigram_k_1_TSNE = TSNE_Process(bigram_k_1_TF_IDF, 'Bigram', 1)
model_data = pd.concat([build_model(bigram_k_1_Model), model_data])
bigram_k_15_TF_IDF, bigram_k_15_features = TF_IDF(raw_data['tweet'], 2, .15)
bigram_k_15_TSNE = TSNE_Process(bigram_k_15_TF_IDF, 'Bigram', '.15')
model_data = pd.concat([build_model(bigram_k_15_TSNE), model_data])

ValueError: could not convert string to float: 'Unigram'

## TSNE Results

TSNE using both unigrams and bigrams seems to have the same results.  A ring with some spread data and a very tight core.  Just looking at some random results from each category, the groupings might be a division of hate speech vs offensive language.  I will try to match these 2 categories and check the performance.  I will only use correct answers from each category.  While categorizing the data is not my eventual goal, I want to see what happens with data this dimensionolly reduced.

### Scores

I used simple Precision, Recall, and F1 Scores.  With this data the bigrams did surprisingly well, with an 82% weighted Precision and a raw accuracy of 70%.  Note:  The weighted scores may be innacurate with no 2 values in the dataset.

### EDA Results

Even though I used an ML algorighim to perform striping, I still consider this EDA as I used basic tequniqes to find a pattern before performing more aggressvie Unstructured Learning.  I found that just paring down the data to 2 columns with bigrams I can create a model with a  70% precision.



In [0]:
color_graph(model_data, "EDA TSNE Results")

In [0]:
def makeScores(d1, d2, text):
    print(text + ":  Raw Precision: " + str(precision_score(d1, d2, average = 'micro')) + \
          ", Weighted Precision: " + str(precision_score(d1, d2, average = 'weighted')) + \
         ", Weighted Recall: " + str(recall_score(d1, d2, average = 'weighted')) +\
         ", Weghted F1: " + str(f1_score(d1, d2, average = 'weighted')))
makeScores(unigram_k_1_Model['predict'], raw_data['class'], "Unigram k = 1")
makeScores(unigram_k_15_Model['predict'], raw_data['class'], "Unigram k = .15")
makeScores(bigram_k_1_Model['predict'], raw_data['class'], "Bigram k = 1")
makeScores(bigram_k_15_Model['predict'], raw_data['class'], "Bigram k = .15")

## Model Building
I'm going to start with a DBSCAN model.  This is a popular model for text categorization and with the large doughnut hole in the middle of the data, I'm going to see if I can get the 2 blocks of data and test the results.
### DBSCAN
DBSCAN or Density-Based Spatial Clustering of Applications with Noise is a clustering algorithm that uses the density of datapoints to classify objects.